<a href="https://colab.research.google.com/github/thekyr955/BI_Data_Assessment/blob/main/BI_Data_Assessment(SQL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q duckdb openpyxl


In [ ]:
import duckdb
import pandas as pd
from google.colab import files


In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]  # Παίρνει το όνομα του αρχείου που ανέβηκε
print(f"Το αρχείο που ανέβηκε: {filename}")


Saving BI_SQL__Modeling_Tests_2025.xlsx to BI_SQL__Modeling_Tests_2025.xlsx
Το αρχείο που ανέβηκε: BI_SQL__Modeling_Tests_2025.xlsx


In [ ]:
## Transform al the excel sheets into tables

In [ ]:
duckdb.sql(f"""
CREATE TABLE financial AS
SELECT * FROM read_xlsx('{filename}', sheet='Financial', header=true);
""")


In [ ]:
duckdb.sql(f"""
CREATE TABLE payment AS
SELECT * FROM read_xlsx('{filename}', sheet='Payment', header=true);
""")


In [ ]:
duckdb.sql(f"""
CREATE TABLE Product AS
SELECT * FROM read_xlsx('{filename}', sheet='Product', header=true);
""")

In [ ]:
duckdb.sql(f"""
CREATE TABLE PaymentType AS
SELECT * FROM read_xlsx('{filename}', sheet='PaymentType', header=true);
""")

In [ ]:
duckdb.sql(f"""
CREATE TABLE PaymentStatus AS
SELECT * FROM read_xlsx('{filename}', sheet='PaymentStatus', header=true);
""")

In [ ]:
duckdb.sql(f"""
CREATE TABLE Customer AS
SELECT * FROM read_xlsx('{filename}', sheet='Customer', header=true);
""")

In [ ]:
duckdb.sql(f"""
CREATE TABLE Brand AS
SELECT * FROM read_xlsx('{filename}', sheet='Brand', header=true);
""")

In [ ]:
duckdb.sql(f"""
CREATE TABLE Device AS
SELECT * FROM read_xlsx('{filename}', sheet='Device', header=true);
""")

In [ ]:
## Question 1
duckdb.sql("""
SELECT count(*) as number_of_cancelled
FROM payment where FK_PAYMENT_STATUS='9F935BEB31030AD0D4D26126C0F39BF2' --Keep only rows where the status equals the cancelled status ID
and month(changed)=01
and year(changed)=2024
""").df()

,number_of_cancelled
0,52


In [ ]:
# Question 2

duckdb.sql("""

SELECT
    SUM(AMOUNT_EUR) AS total_deposit_eur,
    FK_CUSTOMER
FROM payment
WHERE FK_PAYMENT_STATUS = '1F09B6651A28D99473712D885C2C31A3'  -- for completed orders
  AND FK_PAYMENT_TYPE   = 'E2C141E421F9862E778B1F7A0749989F'  -- for deposits
GROUP BY FK_CUSTOMER
ORDER BY total_deposit_eur DESC
LIMIT 10 -- Top 10 customers
""").df()

,total_deposit_eur,FK_CUSTOMER
0,32018.8062,1955ECD86B62AD0ED311EF90B6FBD171
1,29058.6817,946A031C4366696479B05ED47F2C6601
2,25310.6156,F97028B7178D6E1E717E7D76C172013A
3,25235.3141,7823B86D08287DCFCAB3A3C9DF9CEBA8
4,5955.0688,100F4BB0E89B2FF9B90BE892381423B0
5,5060.8445,561BD4CE244ECAD2BC19CE7CE11E7C68
6,4201.7942,30FB19AC94654E934312DFB4AD4DF214
7,3235.4889,B588C8C794347A79337754889F4D92F9
8,2789.6597,49249690814FC6B5029E28F790F0B761
9,2193.0958,932BD5DE6BD29264BF0F3D98B650561A


In [ ]:
## Question 3
duckdb.sql("""
WITH WITHDRAWAL AS (
  -- Total withdrawals per brand
  SELECT SUM(amount_eur) AS withdr,
         BRAND_NAME
  FROM payment
  INNER JOIN Customer ON SK_CUSTOMER = fk_customer
  INNER JOIN brand ON fk_brand = sk_brand
  WHERE FK_PAYMENT_TYPE = 'DD4FD7A8FF1872E56DA2312CF7143E83'
  GROUP BY BRAND_NAME
),
deposits AS (
  -- Total deposits per brand
  SELECT SUM(amount_eur) AS dep,
         BRAND_NAME
  FROM payment
  INNER JOIN Customer ON SK_CUSTOMER = fk_customer
  INNER JOIN brand ON fk_brand = sk_brand
  WHERE FK_PAYMENT_TYPE = 'E2C141E421F9862E778B1F7A0749989F'
  GROUP BY BRAND_NAME
)
-- Net cash flow = deposits - withdrawals
SELECT (dep - withdr) AS Net_Cash_Flow,
       WITHDRAWAL.brand_name
FROM WITHDRAWAL
INNER JOIN deposits ON deposits.BRAND_NAME = WITHDRAWAL.BRAND_NAME
ORDER BY 1 DESC
limit 1 -- the first for the largest amount
""").df()

,Net_Cash_Flow,BRAND_NAME
0,219122.1007,BetssonB


In [ ]:
# Question 4
duckdb.sql("""
-- Total turnover per device/channel
SELECT
    SUM(TURNOVER) AS total_turnover,
    d.CHANNEL      -- channel name
FROM financial f
INNER JOIN device d
  ON f.FK_DEVICE = d.SK_DEVICE
GROUP BY d.CHANNEL
ORDER BY total_turnover DESC
limit 1 -- we want the first
""").df()

,total_turnover,CHANNEL
0,619143.972252,Mobile


In [ ]:
##  Question 5

duckdb.sql(""" -- Top 3 highest payments per customer (only final statuses)
WITH base AS (
  -- Keep only final payment records
  SELECT DISTINCT
    p.FK_CUSTOMER,
    CAST(p.AMOUNT_EUR AS DOUBLE) AS AMOUNT_EUR,
    p.CHANGED
  FROM Payment p
  JOIN PaymentStatus s
    ON p.FK_PAYMENT_STATUS = s.SK_PAYMENT_STATUS
  WHERE s.IS_FINAL_STATUS = TRUE
),
ranked AS ( -- Rank payments per customer by amount (and date as tie-breaker)

  SELECT
    FK_CUSTOMER,
    AMOUNT_EUR,
    ROW_NUMBER() OVER (
      PARTITION BY FK_CUSTOMER
      ORDER BY AMOUNT_EUR DESC, CHANGED DESC
    ) AS rn
  FROM base
)
-- Return only the top 3 payments for each customer
SELECT FK_CUSTOMER, AMOUNT_EUR, rn
FROM ranked
WHERE rn <= 3
ORDER BY FK_CUSTOMER, rn;
""").df()


,FK_CUSTOMER,AMOUNT_EUR,rn
0,100F4BB0E89B2FF9B90BE892381423B0,576.1761,1
1,100F4BB0E89B2FF9B90BE892381423B0,321.3218,2
2,100F4BB0E89B2FF9B90BE892381423B0,242.8946,3
3,182D7995C3B327C3C9868EB77575B9FC,3.3361,1
4,1955ECD86B62AD0ED311EF90B6FBD171,1532.2943,1
5,1955ECD86B62AD0ED311EF90B6FBD171,1458.2496,2
6,1955ECD86B62AD0ED311EF90B6FBD171,1097.6037,3
7,25191C5DB406EDD1A52E31481A55FBC2,83.8349,1
8,25191C5DB406EDD1A52E31481A55FBC2,78.4067,2
9,25191C5DB406EDD1A52E31481A55FBC2,67.6542,3


In [ ]:
## Question 6
duckdb.sql("""
-- Totals by Device, by Date, and by (Device x Date)
WITH base AS (
  SELECT
    d.CHANNEL                       AS device,
    CAST(f.DATE_FROM AS DATE)       AS dt,
    f.TURNOVER,
    f.GGR
  FROM financial f
  JOIN device d
    ON f.FK_DEVICE = d.SK_DEVICE
)
SELECT
  device,
  dt,
  SUM(TURNOVER) AS total_turnover,
  SUM(GGR)      AS total_ggr,
  CASE
    WHEN device IS NOT NULL AND dt IS NOT NULL THEN 'Device x Date'
    WHEN device IS NOT NULL AND dt IS NULL     THEN 'Device Total'
    WHEN device IS NULL     AND dt IS NOT NULL THEN 'Date Total'
    ELSE 'Grand Total'
  END AS grain
FROM base
GROUP BY GROUPING SETS (
  (device, dt),
  (device),
  (dt)

)
ORDER BY
  grain,
  device NULLS LAST,
  dt NULLS LAST;

""").df()

,device,dt,total_turnover,total_ggr,grain
0,None,2024-01-01,4993.769300,428.954300,Date Total
1,None,2024-01-02,900.413200,197.374500,Date Total
2,None,2024-01-03,1196.088500,316.882500,Date Total
3,None,2024-01-04,384.233301,53.453601,Date Total
4,None,2024-01-05,127.571000,12.861200,Date Total
...,...,...,...,...,...
1703,Web,2025-07-27,174.926100,-15.343300,Device x Date
1704,Web,2025-07-28,335.513600,31.537500,Device x Date
1705,Web,2025-07-29,287.695300,2.049000,Device x Date
1706,Web,2025-07-30,45.512320,40.952220,Device x Date


In [ ]:
# Question 7
duckdb.sql("""
select ggr,
turnover-win,
case when ggr=(turnover-win) then 'Equal' else 'Not Equal' end as check
from financial""").df()

,GGR,(turnover - win),check
0,-20.521700,296.880900,Not Equal
1,0.000000,0.000000,Equal
2,0.077900,0.270100,Not Equal
3,0.000000,0.000000,Equal
4,17.233500,398.719100,Not Equal
...,...,...,...
6031,0.000000,0.000000,Equal
6032,23.723212,118.806714,Not Equal
6033,33.117700,237.918300,Not Equal
6034,0.000000,0.000000,Equal


In [ ]:
# Question 8
duckdb.sql("""
-- Find account status changes per customer
WITH ordered AS (
  -- Order each customer's status history chronologically
  SELECT
    c.CUSTOMER_ID,
    c.STATUS,
    CAST(c.STATUS_START_DATE AS DATE) AS status_start_date,
    CAST(c.STATUS_END_DATE   AS DATE) AS status_end_date,
    LAG(c.STATUS) OVER (
      PARTITION BY c.CUSTOMER_ID
      ORDER BY CAST(c.STATUS_START_DATE AS DATE)
    ) AS prev_status
  FROM Customer c
),
changes AS (
  -- Keep only rows where the status actually changed
  SELECT
    CUSTOMER_ID,
    prev_status      AS previous_status,      -- status before the change
    STATUS           AS new_status,           -- status after the change
    status_start_date AS change_date          -- date when new status started
  FROM ordered
  WHERE prev_status IS NOT NULL
    AND prev_status <> STATUS
)
-- Final result: one row per status change
SELECT
  CUSTOMER_ID,
  previous_status,
  new_status,
  change_date
FROM changes
ORDER BY CUSTOMER_ID, change_date;

""").df()

,CUSTOMER_ID,previous_status,new_status,change_date
0,8C51FDB8-8365-11E2-A8BE-1CC1DEE5EBDC,Active,Inactive,2024-08-06
1,FFDA7D52-BD2F-4924-A1A7-522E16362AEE,Active,Closed,2024-06-30


In [ ]:
# Question 9
duckdb.sql("""
select fk_customer,
sum(turnover),
sum(ngr)
from financial
group by fk_customer
having sum(turnover)>=400000
""").df()

,FK_CUSTOMER,sum(turnover),sum(ngr)
0,1955ecd86b62ad0ed311ef90b6fbd171,484159.2085,14190.204702


In [ ]:
# Question 10
duckdb.sql("""
select fk_customer,
sum(turnover),
sum(ngr)
from financial
group by fk_customer
having sum(turnover)>=400000""").df()

,FK_CUSTOMER,sum(turnover),sum(ngr)
0,1955ecd86b62ad0ed311ef90b6fbd171,484159.2085,14190.204702


In [ ]:
# Question 11
duckdb.sql("""
select max(changed) last_withdrawal_date,
FK_CUSTOMER from payment
where FK_PAYMENT_TYPE='DD4FD7A8FF1872E56DA2312CF7143E83' --withdrawal
group by FK_CUSTOMER
""").df()

,last_withdrawal_date,FK_CUSTOMER
0,2025-02-04,100F4BB0E89B2FF9B90BE892381423B0
1,2025-07-16,1955ECD86B62AD0ED311EF90B6FBD171
2,2025-07-28,30FB19AC94654E934312DFB4AD4DF214
3,2025-06-17,49249690814FC6B5029E28F790F0B761
4,2025-07-17,6E04CBD85AAC91C40044976AC179BC7D
5,2025-07-13,7823B86D08287DCFCAB3A3C9DF9CEBA8
6,2025-03-30,932BD5DE6BD29264BF0F3D98B650561A
7,2024-11-04,CD75B46599395D35946C1328FC167ABC
8,2024-07-11,EEC92EDAB54F84FF0932CFD2CE7E0B09
9,2024-11-14,EECF153A439A7EDBA167A681F2991D74


In [ ]:
# 12 Question
duckdb.sql("""
SELECT
    p.FK_CUSTOMER AS sk_customer,
    MIN(p.CHANGED) AS first_deposit_date
FROM Payment p
WHERE p.FK_PAYMENT_TYPE = 'E2C141E421F9862E778B1F7A0749989F' --deposit
  AND p.FK_PAYMENT_STATUS = '1F09B6651A28D99473712D885C2C31A3' --captured
GROUP BY p.FK_CUSTOMER
ORDER BY first_deposit_date;
         """).df()

,sk_customer,first_deposit_date
0,F97028B7178D6E1E717E7D76C172013A,2024-01-01
1,561BD4CE244ECAD2BC19CE7CE11E7C68,2024-01-02
2,B588C8C794347A79337754889F4D92F9,2024-01-02
3,100F4BB0E89B2FF9B90BE892381423B0,2024-01-04
4,6E04CBD85AAC91C40044976AC179BC7D,2024-01-04
5,932BD5DE6BD29264BF0F3D98B650561A,2024-01-04
6,7823B86D08287DCFCAB3A3C9DF9CEBA8,2024-01-06
7,B57868A1FB16267D79B891ADFF7CAD89,2024-01-12
8,25191C5DB406EDD1A52E31481A55FBC2,2024-02-03
9,30FB19AC94654E934312DFB4AD4DF214,2024-02-16
